In [4]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
import os

Charger les données de DVF

In [41]:
#les données suivantes sont les données dans herault  (34)
url0 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2024/departements/34.csv.gz" # 2024 (seulement Q1)
url1 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/departements/34.csv.gz" # 2023
url2 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2022/departements/34.csv.gz" # 2022
url3 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2021/departements/34.csv.gz" # 2021
url4 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2020/departements/34.csv.gz" # 2020
url5 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2019/departements/34.csv.gz" # 2019

In [42]:
# Load data from URLs
def load_data(url):
    
    df = pd.read_csv(url,compression="gzip", low_memory=False)  
    df.dropna(axis=1, how='all', inplace=True)
    df = df.dropna(subset=['code_postal']) # enlever les valeurs nulles de code postal
    df['code_postal'] = df['code_postal'].astype(int).astype(str)
    #code_mask = df['code_postal'].isin(codes_postaux_metropole) # filtre code postal pour Montpellier Méditerranée Métropole
    #df =df[code_mask]
    df['date_mutation']= pd.to_datetime(df['date_mutation'], format='%Y-%m-%d', errors='coerce')
    df['year'] = df['date_mutation'].dt.year
    return df
    
df0 = load_data(url0)
df1 = load_data(url1)
df2 = load_data(url2)
df3 = load_data(url3)
df4 = load_data(url4)
df5 = load_data(url5)

In [43]:
df0.head(2)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,year
0,2024-179483,2024-01-03,1,Vente,500000.0,21.0,NaN,RUE DE LA BERGERIE DE CAUNELLE,0009,34990,...,127.0,4.0,S,sols,NaN,NaN,638.0,3.800856,43.616081,2024
1,2024-179483,2024-01-03,1,Vente,500000.0,21.0,NaN,RUE DE LA BERGERIE DE CAUNELLE,0009,34990,...,NaN,0.0,S,sols,NaN,NaN,638.0,3.800856,43.616081,2024


In [44]:
df0.shape[0], df1.shape[0], df2.shape[0], df3.shape[0], df4.shape[0], df5.shape[0]  

(30085, 77463, 94972, 95505, 73220, 39385)

Combine les 5 fichiers

In [45]:
df_combined = pd.concat([df0, df1, df2, df3, df4, df5], ignore_index=True)

In [46]:
df_combined.shape[0]

410630

In [47]:
select_columns = ['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'numero_volume', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales']
df_s = df_combined[select_columns]

In [48]:
df12= df_s[df_s['lot2_surface_carrez'].notnull()]  #| df['latitude'].isnull()]
df12.head(2)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,numero_volume,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales
30,2024-179499,2024-01-04,1,Vente,118000.0,0241,34400,34145,Lunel,34,NaN,NaN,51.58,NaN,NaN,NaN,2.0,Appartement,52.0,2.0
31,2024-179499,2024-01-04,1,Vente,118000.0,0241,34400,34145,Lunel,34,NaN,NaN,51.58,NaN,NaN,NaN,3.0,Dépendance,NaN,0.0


In [49]:
df_combined.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'numero_volume',
       'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'year', 'ancien_code_commune', 'ancien_nom_commune',
       'ancien_id_parcelle'],
      dtype='object')

In [50]:
# Reserve les colonnes suivantes
columns_to_keep = [
    'valeur_fonciere',               
    'type_local',                   
    'surface_reelle_bati',           
    'surface_terrain',               
    'nombre_pieces_principales',     
    'nombre_lots',                  
    'code_postal',                  
    'nom_commune',                   
    'code_commune',                  
    'code_departement',              
    'longitude',                     
    'latitude',                      
    'date_mutation',
    'nature_mutation',                 
    'year',                        
]

# filtre les colonnes
df_filtered = df_combined[columns_to_keep]

In [51]:
df_filtered.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024
1,500000.0,Dépendance,NaN,638.0,0.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024


In [52]:
df_grouped = df_filtered.groupby('type_local').size()
df_grouped

type_local
Appartement                                  83245
Dépendance                                  110082
Local industriel. commercial ou assimilé     13726
Maison                                       63485
dtype: int64

In [53]:
df_type_local = df_filtered[df_filtered['type_local'] .isin(['Appartement','Maison'])]
df_type_local.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024
3,40000.0,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,Vente,2024
8,263660.0,Maison,65.0,NaN,3.0,1,34090,Montpellier,34172,34,3.852466,43.638047,2024-01-05,Vente,2024
11,160500.0,Appartement,43.0,NaN,2.0,2,34090,Montpellier,34172,34,3.868303,43.636036,2024-01-05,Vente,2024
13,250000.0,Appartement,66.0,NaN,3.0,2,34570,Pignan,34202,34,3.770587,43.586034,2024-01-08,Vente,2024


Dépendance : Garage / Cave / Grenier / Abri de Jardin / Piscine / Atelier 
Local industriel, commercial ou assimilé : Local industriel / Local commercial / Asimilé  

In [54]:
df_grouped = df_filtered.groupby('nature_mutation').size()
df_grouped

nature_mutation
Adjudication                            1203
Echange                                 2593
Expropriation                             13
Vente                                 369219
Vente en l'état futur d'achèvement     35971
Vente terrain à bâtir                   1631
dtype: int64

In [55]:
print(df_type_local.columns)


Index(['valeur_fonciere', 'type_local', 'surface_reelle_bati',
       'surface_terrain', 'nombre_pieces_principales', 'nombre_lots',
       'code_postal', 'nom_commune', 'code_commune', 'code_departement',
       'longitude', 'latitude', 'date_mutation', 'nature_mutation', 'year'],
      dtype='object')


On va garder : Vente /  Vente en l'état futur d'achèvement /Vente terrain à bâtir   

In [ ]:
df_vente = df_type_local[df_type_local['nature_mutation'].isin([
    'Vente', 
    'Vente en l\'état futur d\'achèvement', 
    'Vente terrain à bâtir'
])]
df_vente.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024
3,40000.0,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,Vente,2024
8,263660.0,Maison,65.0,NaN,3.0,1,34090,Montpellier,34172,34,3.852466,43.638047,2024-01-05,Vente,2024
11,160500.0,Appartement,43.0,NaN,2.0,2,34090,Montpellier,34172,34,3.868303,43.636036,2024-01-05,Vente,2024
13,250000.0,Appartement,66.0,NaN,3.0,2,34570,Pignan,34202,34,3.770587,43.586034,2024-01-08,Vente,2024


In [57]:
#vérifie les valeurs nulles dans chaque colonne
missing_values_per_column = df_vente.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 53
type_local                       0
surface_reelle_bati              2
surface_terrain              74652
nombre_pieces_principales        2
nombre_lots                      0
code_postal                      0
nom_commune                      0
code_commune                     0
code_departement                 0
longitude                     1771
latitude                      1771
date_mutation                    0
nature_mutation                  0
year                             0
dtype: int64


In [58]:
# total des valeurs manquantes
total_missing_values = df_vente.isnull().sum().sum()
print(f"Total missing values: {total_missing_values}")

Total missing values: 78251


In [59]:
df_longitude_latitude = df_vente[df_vente['longitude'].isnull()|df_vente['latitude'].isnull()]

df_longitude_latitude

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
2097,6500.0,Maison,45.0,NaN,3.0,0,34700,Les Plans,34205,34,NaN,NaN,2024-01-29,Vente,2024
3706,560000.0,Maison,43.0,195.0,2.0,0,34000,Montpellier,34172,34,NaN,NaN,2024-02-20,Vente,2024
5000,8000.0,Maison,178.0,1152.0,6.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-02-29,Vente,2024
5003,8000.0,Maison,178.0,1200.0,6.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-02-29,Vente,2024
7477,400000.0,Maison,107.0,1133.0,3.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-03-14,Vente,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409228,180000.0,Appartement,34.0,NaN,2.0,1,34300,Agde,34003,34,NaN,NaN,2019-12-06,Vente,2019
409402,60000.0,Appartement,34.0,NaN,2.0,1,34300,Agde,34003,34,NaN,NaN,2019-11-29,Vente,2019
409710,530000.0,Maison,144.0,878.0,5.0,0,34120,Castelnau-de-Guers,34056,34,NaN,NaN,2019-12-06,Vente,2019
410049,103000.0,Appartement,32.0,NaN,2.0,1,34340,Marseillan,34150,34,NaN,NaN,2019-12-16,Vente,2019


In [60]:
df_vente.dtypes

valeur_fonciere                     float64
type_local                           object
surface_reelle_bati                 float64
surface_terrain                     float64
nombre_pieces_principales           float64
nombre_lots                           int64
code_postal                          object
nom_commune                          object
code_commune                          int64
code_departement                      int64
longitude                           float64
latitude                            float64
date_mutation                datetime64[ns]
nature_mutation                      object
year                                  int64
dtype: object

In [61]:
df_popu = pd.read_csv('herault_population_2019_2024.csv')
df_popu.head()

,code_commune,year,population
0,34091,2024,56
1,34091,2021,54
2,34092,2024,967
3,34092,2023,977
4,34093,2025,147


In [62]:

df_vente['code_postal'] = df_vente['code_postal'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_10196\3313786434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vente['code_postal'] = df_vente['code_postal'].astype(int)


In [63]:
#merge df_popu + de_vente
df_merged = pd.merge(df_vente, df_popu, on=['code_commune', 'year'], how='inner')

In [64]:
df_merged = df_merged.drop(columns=['code_departement', 'nature_mutation'])

In [65]:
df_merged['nombre_pieces_principales'] = df_merged['nombre_pieces_principales'].fillna(0)

In [67]:
df_merged['nombre_pieces_principales'] = df_merged['nombre_pieces_principales'].astype(int)

In [69]:
df_merged.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population
0,500000.0,Maison,127.0,638.0,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344
1,144000.0,Appartement,47.0,NaN,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344


In [70]:
df_rev = pd.read_csv('herault_revenu.csv')
df_rev.head()

,code_commune,rev_fisc_moyen_reference
0,34002,21.558788
1,34004,17.478192
2,34006,19.962507
3,34007,17.640876
4,34011,22.605179


In [71]:
#merge df_rev + de_merged
df_merged_rev = pd.merge(df_merged, df_rev, on=['code_commune'], how='inner')

In [87]:
df_merged_rev.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population,rev_fisc_moyen_reference
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344,28.72
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344,28.72


In [73]:
df_merged_rev['rev_fisc_moyen_reference'] = df_merged_rev['rev_fisc_moyen_reference'].round(2)

In [74]:
df_merged_rev.shape[0]

129940

In [75]:

missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 52
type_local                       0
surface_reelle_bati              1
surface_terrain              66497
nombre_pieces_principales        0
nombre_lots                      0
code_postal                      0
nom_commune                      0
code_commune                     0
longitude                     1414
latitude                      1414
date_mutation                    0
year                             0
population                       0
rev_fisc_moyen_reference        10
dtype: int64


In [76]:
# Delete rows where 'valeur_fonciere' is null
df_merged_rev = df_merged_rev.dropna(subset=['valeur_fonciere'])

In [77]:
df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].fillna(0)

C:\Users\yutin\AppData\Local\Temp\ipykernel_10196\491585317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].fillna(0)


In [85]:
df_merged_rev.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population,rev_fisc_moyen_reference
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344,28.72
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344,28.72


In [79]:
df_merged_rev = df_merged_rev.dropna(subset=['surface_reelle_bati'])

In [80]:
df_merged_rev['surface_reelle_bati'] = df_merged_rev['surface_reelle_bati'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_10196\391343707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_rev['surface_reelle_bati'] = df_merged_rev['surface_reelle_bati'].astype(int)


In [81]:
df_merged_rev['valeur_fonciere'] = df_merged_rev['valeur_fonciere'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_10196\2417924950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_rev['valeur_fonciere'] = df_merged_rev['valeur_fonciere'].astype(int)


In [83]:
df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_10196\1582057432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].astype(int)


In [99]:
df_merged_rev.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72


In [88]:
df_merged_rev = df_merged_rev.drop(columns=['date_mutation'])

In [89]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 0
type_local                      0
surface_reelle_bati             0
surface_terrain                 0
nombre_pieces_principales       0
nombre_lots                     0
code_postal                     0
nom_commune                     0
code_commune                    0
longitude                    1413
latitude                     1413
year                            0
population                      0
rev_fisc_moyen_reference       10
dtype: int64


In [98]:
df_merged_rev.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,43.617937,2024,11344,28.72
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,43.613413,2024,11344,28.72
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,43.613778,2024,11344,28.72


In [92]:
#imputation longitud et latitude par la moyenne 

# Group by 'code_commune' and fill missing 'longitude' and 'latitude' with the group's mean
df_merged_rev['longitude'] = df_merged_rev.groupby('code_commune')['longitude'].transform(lambda x: x.fillna(x.mean()))
df_merged_rev['latitude'] = df_merged_rev.groupby('code_commune')['latitude'].transform(lambda x: x.fillna(x.mean()))


In [93]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere               0
type_local                    0
surface_reelle_bati           0
surface_terrain               0
nombre_pieces_principales     0
nombre_lots                   0
code_postal                   0
nom_commune                   0
code_commune                  0
longitude                     0
latitude                      0
year                          0
population                    0
rev_fisc_moyen_reference     10
dtype: int64


Il reste 10 valeur nulles dans rev_fisc_moyen_reference.

In [94]:
# calcule le moyen de 'rev_fisc_moyen_reference
mean_value = df_merged_rev['rev_fisc_moyen_reference'].mean()

# fillna le moyen de 'rev_fisc_moyen_reference
df_merged_rev['rev_fisc_moyen_reference'] = df_merged_rev['rev_fisc_moyen_reference'].fillna(mean_value)


In [96]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
dtype: int64


Il n'y a plus de valeur nulles. 

In [103]:
df_cho = pd.read_csv('taux_chomage.csv')
df_cho.head()

,code_commune,unemployment_rate
0,34001,25.75
1,34002,27.99
2,34003,39.01
3,34004,4.55
4,34005,51.96


In [104]:
#merge df_merged_rev + dedf_cho
df_merged_cho = pd.merge(df_merged_rev, df_cho, on=['code_commune'], how='inner')

In [105]:
df_merged_cho

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72,28.32
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72,28.32
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,43.617937,2024,11344,28.72,28.32
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,43.613413,2024,11344,28.72,28.32
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,43.613778,2024,11344,28.72,28.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129882,355000,Maison,123,767,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00
129883,355000,Maison,123,1000,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00
129884,355000,Maison,123,500,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00
129885,236000,Maison,45,405,2,0,34800,Mérifons,34156,3.284967,43.620958,2020,46,25.85,0.00


In [107]:
df_merged_cho.to_csv("projet_immo.csv", index=False)

In [108]:
missing_values_per_column = df_merged_cho.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
unemployment_rate            1
dtype: int64


In [112]:
valeurnulles = df_merged_cho[df_merged_cho['unemployment_rate'].isnull()]
valeurnulles

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate
129881,452860,Maison,105,81,3,0,34520,Saint-Félix-de-l'Héras,34253,3.290564,43.836364,2021,35,24.04,NaN


In [113]:
# il ny a pas de chomeur dans cette commune 34253, donc on met 0  
df_merged_cho['unemployment_rate'] = df_merged_cho['unemployment_rate'].fillna(0)


In [114]:
missing_values_per_column = df_merged_cho.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
unemployment_rate            0
dtype: int64
